# An Agentic RAG Workflow Using a Query Router Built with LangGraph

## 1. Setup and Imports

In [78]:
# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [112]:
import os

from langchain_community.retrievers import WikipediaRetriever
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents.base import Document
from langchain_openai import ChatOpenAI 

from langgraph.graph import START, END, StateGraph

In [129]:
# set environment variables (please replace with your own)
os.environ["OPENAI_API_KEY"] = ""
os.environ["TAVILY_API_KEY"] = ""

## 2. Building a Simple RAG Pipeline with LangGraph with Wikipedia Retriever

Rather than using LangChain's pre-packaged chains that integrate both retrieval and generation steps, we will deconstruct the RAG pipeline into a graph-based workflow utilizing LangGraph. This approach involves creating separate nodes for retrieval and generation processes.

In [82]:
# define state of class for the graph
from typing import TypedDict, List

# defining the state of our graph
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:     
        query: A string representing the user's query.
        retrieved_docs: A list of Document objects retrieved from the Wikipedia retriever.
        answer: A string representing the final answer to the user's query.
    """
    query: str
    retrieved_docs: List[Document]
    answer: str

In [83]:
# testing the wikipedia retriever
wikipedia_retriever = WikipediaRetriever()
wikipedia_retriever.invoke("When was manchester united founded?")

[Document(metadata={'title': 'Manchester United F.C.', 'summary': "Manchester United Football Club, commonly referred to as Man United (often stylised as Man Utd) or simply United, is a professional football club based in Old Trafford, Greater Manchester, England. They compete in the Premier League, the top tier of English football. Nicknamed the Red Devils, they were founded as Newton Heath LYR Football Club in 1878, but changed their name to Manchester United in 1902. After a spell playing in Clayton, Manchester, the club moved to their current stadium, Old Trafford, in 1910.\nDomestically, Manchester United have won a joint-record twenty top-flight league titles, thirteen FA Cups, six League Cups and a record twenty-one FA Community Shields. Additionally, in international football, they have won the European Cup/UEFA Champions League three times, and the UEFA Europa League, the UEFA Cup Winners' Cup, the UEFA Super Cup, the Intercontinental Cup and the FIFA Club World Cup once each.

In [84]:
# encapsulating the wikipedia retriever into a node
def retrieve_from_wikipedia(state: GraphState) -> GraphState:
    """
    Retrieves documents from Wikipedia based on the query.

    Args:
        state: A dictionary containing the state of the graph.

    Returns:
        Updated state with retrieved documents.
    """
    print("*** Running Node: Retrieve from Wikipedia ***")
    retrieved_docs = wikipedia_retriever.invoke(state["query"])
    return {"retrieved_docs": retrieved_docs}

In [85]:
rag_prompt = """You are an AI  assistant. Your main task is to answer questions people may have about Sajal.
Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {query}
Context: {context}
Answer:
"""

rag_prompt_template = ChatPromptTemplate.from_template(rag_prompt)

llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [86]:
generation_answer_chain = rag_prompt_template | llm | StrOutputParser()
def generate_answer_with_retrieved_documents(state: GraphState) -> GraphState:
  """Node to generate answer using retrieved documents"""
  print("*** Running Node: Generate Answer with Retrieved Documents ***") 
  query = state["query"]
  documents = state["retrieved_docs"]
  answer = generation_answer_chain.invoke({"query": query, "context": documents})
  return {"answer": answer}

In [87]:
# building a graph
def compile_graph():
    workflow = StateGraph(GraphState)
    ### add the nodes
    workflow.add_node("retrieve_wikipedia", retrieve_from_wikipedia)
    workflow.add_node("generate_answer", generate_answer_with_retrieved_documents)
    ## build graph
    workflow.set_entry_point("retrieve_wikipedia")
    workflow.add_edge("retrieve_wikipedia", "generate_answer")
    workflow.add_edge("generate_answer", END)
    ## compile graph
    return workflow.compile()

In [ ]:
app = compile_graph()
def response_from_graph(query: str):
    return app.invoke({"query": query})["answer"]

In [89]:
print(response_from_graph("When was manchester united founded?"))

*** Running Node: Retrieve from Wikipedia ***
*** Running Node: Generate Answer with Retrieved Documents ***
Manchester United was founded as Newton Heath LYR Football Club in 1878.


In [ ]:
# Try a query that relies on up-to-date information
print(response_from_graph("Who are Manchester United looking to sign next season?"))

*** Running Node: Retrieve from Wikipedia ***
*** Running Node: Generate Answer with Retrieved Documents ***
I don't know who Manchester United is looking to sign next season, as the provided context does not include information about their transfer targets.


As seen above, the Wikipedia retriever is not able to retrieve up-to-date information, and our RAG pipeline suffers from it.

## 3. Agentic RAG Pipeline with Router and Web Search Retriever

Start with adding a web search retriever using Tavily API

In [ ]:
from langchain_community.retrievers import TavilySearchAPIRetriever

tavily_retriever = TavilySearchAPIRetriever(k=3)
# test out the tavily retriever
tavily_retriever.invoke("Who are Manchester United looking to sign next season?")

[Document(metadata={'title': "Man United's next signings once Matheus Cunha and Sandro Tonali ...", 'source': 'https://www.manchestereveningnews.co.uk/sport/football/transfer-news/man-uniteds-next-signings-once-31614635', 'score': 0.71496695, 'images': []}, page_content="Ruben Amorim needs a new centre-forward for Manchester United A striker is simply the priority. Ipswich Town's Liam Delap is understood to be the number one target with his £30m release clause"),
 Document(metadata={'title': 'Man Utd to sign THREE attackers with choice between Semenyo and Mbeumo made', 'source': 'https://www.teamtalk.com/manchester-united/man-utd-choose-between-mbeumo-semenyo-third-attacker-signing', 'score': 0.564027, 'images': []}, page_content='Manchester United intend to make three major additions to their forward line, and with deals one and two already advancing, a the club have now chosen between Antoine Semenyo and Bryan Mbeumo for'),
 Document(metadata={'title': 'Who will Man Utd sign this sum

In [101]:
# create a node for the web search retriever    
def retrieve_from_web_search(state: GraphState) -> GraphState:
    """
    Retrieves documents from web search based on the query.

    Args:
        state: A dictionary containing the state of the graph.

    Returns:
        Updated state with retrieved documents.
    """
    print("*** Running Node: Retrieve from Web Search ***")
    retrieved_docs = tavily_retriever.invoke(state["query"])
    return {"retrieved_docs": retrieved_docs}

In [ ]:
# create an llm chain which determines which retriever to use
from pydantic import BaseModel, Field

class RouterOutput(BaseModel):
    """Schema for router output"""
    chosen_retriever: str = Field(description="The name of the chosen retriever. Either 'wikipedia' or 'web_search'")
    

router_prompt = """
You are a helpful assistant that can determine which retriever to use based on the query.
If a given query is about a topic based on historical context, output "wikipedia". 
If a given query is about a topic based on current events, output "web_search".    

Query: {query}
"""

router_prompt_template = PromptTemplate.from_template(router_prompt)
llm_with_router_output = llm.with_structured_output(RouterOutput)
router_chain = router_prompt_template | llm_with_router_output

In [ ]:
# test out the router chain
router_chain.invoke({"query": "What is Manchester United?"})

RouterOutput(chosen_retriever='wikipedia')

In [108]:
router_chain.invoke({"query": "Who are Manchester United looking to sign next season?"})

RouterOutput(chosen_retriever='web_search')

In [118]:
# defining the state of our graph
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:     
        query: A string representing the user's query.
        retrieved_docs: A list of Document objects retrieved from the Wikipedia retriever.
        answer: A string representing the final answer to the user's query.
    """
    chosen_retriever: str
    query: str
    retrieved_docs: List[Document]
    answer: str

In [119]:
# create a router node to determine which retriever to use.
def query_router(state: GraphState) -> GraphState:
    """
    Determines which retriever to use based on the query.

    Args:
        state: A dictionary containing the state of the graph.

    Returns:
        Updated state with retrieved documents.
    """
    print("*** Running Node: Query Router ***")
    chosen_retriever = router_chain.invoke({"query": state["query"]}).chosen_retriever
    print(f"Chosen retriever: {chosen_retriever}")
    return {"chosen_retriever": chosen_retriever} 

In [ ]:
# define routing function which will be used in the conditional edge
def routing_function(state: GraphState) -> str:
    """Conditional edge for the routing function which decides the next node to execute."""
    return state["chosen_retriever"]

In [122]:
# create a new graph with the query router
def compile_agentic_rag_graph():
    workflow = StateGraph(GraphState)
    ### add the nodes
    workflow.add_node("query_router", query_router)
    workflow.add_node("retrieve_wikipedia", retrieve_from_wikipedia)
    workflow.add_node("retrieve_web_search", retrieve_from_web_search)
    workflow.add_node("generate_answer", generate_answer_with_retrieved_documents)
    ## build graph
    workflow.set_entry_point("query_router")
    workflow.add_conditional_edges(
        "query_router",
        routing_function,
        {
            "wikipedia": "retrieve_wikipedia",
            "web_search": "retrieve_web_search"
        }
    )
    workflow.add_edge("retrieve_wikipedia", "generate_answer")
    workflow.add_edge("retrieve_web_search", "generate_answer") 
    workflow.add_edge("generate_answer", END)
    ## compile graph
    return workflow.compile()

In [123]:
app = compile_agentic_rag_graph()
def response_from_graph(query: str):
    return app.invoke({"query": query})["answer"]

In [124]:
print(response_from_graph("When was Manchester United incorporated?"))

*** Running Node: Query Router ***
Chosen retriever: wikipedia
*** Running Node: Retrieve from Wikipedia ***
*** Running Node: Generate Answer with Retrieved Documents ***
Manchester United was incorporated in 1902 when the club changed its name from Newton Heath LYR Football Club to Manchester United.


In [127]:
print(response_from_graph("Who are Manchester United looking to sign next?"))

*** Running Node: Query Router ***
Chosen retriever: web_search
*** Running Node: Retrieve from Web Search ***
*** Running Node: Generate Answer with Retrieved Documents ***
Manchester United are looking to sign Ipswich Town's Liam Delap, who is understood to be their number one target with a £30m release clause.


The query that was previously not answered correctly is now answered correctly!